# Load packages and data

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [5]:
combined_data_2point5 = pd.read_excel("../data/combined_data_2point5.xlsx")
pricedat2 = pd.read_excel("../data/pricedat2.xlsx")

In [6]:
# Keep the first 137 columns
combined_data_2point5 = combined_data_2point5.iloc[:, :137]

In [4]:
# Filter the DataFrame to keep rows where 'Issue' is <= 3
# combined_data_2point5 = combined_data_2point5[combined_data_2point5['Issue'] <= 3]

# Clean data

## Demographics

In [7]:
import pandas as pd
import numpy as np

# Correct misspelled DayofWeek values
combined_data_2point5.loc[combined_data_2point5['DayofWeek'] == 'Tueday', 'DayofWeek'] = 'Tuesday'
combined_data_2point5.loc[combined_data_2point5['DayofWeek'] == 'Modnay', 'DayofWeek'] = 'Monday'

# Categorize 'length' into 'length2'
combined_data_2point5['length2'] = pd.cut(
    combined_data_2point5['length'],
    bins=[-float('inf'), 5, 15, 30, float('inf')],
    labels=[1, 2, 3, 4]
).astype(float)  # Ensure it's numeric

# Drop specific observations by 'responseid'
combined_data_2point5 = combined_data_2point5[~combined_data_2point5['ResponseID'].isin(['R_79wmeJJXedp5Abj', 'R_eWcombined_data_2point5zOhU3llfEHz'])]

# Map Q52 to 'age'
combined_data_2point5['age'] = combined_data_2point5['Q52'].replace({1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 6})

# Categorize Q61 into 'edu'
combined_data_2point5['edu'] = pd.cut(
    combined_data_2point5['Q61'],
    bins=[-float('inf'), 2, 4, float('inf')],
    labels=[1, 2, 3]
).astype(float)

# Assign 'gender' from Q51
combined_data_2point5['gender'] = combined_data_2point5['Q51']

# Map Q64 to 'race'
combined_data_2point5['race'] = pd.cut(
    combined_data_2point5['Q64'],
    bins=[-float('inf'), 1, 2, 3, float('inf')],
    labels=[1, 2, 3, 4]
).astype(float)

# Define a mapping for Q60 to region, state, and tz
Q60_mapping = {
    1: ('Alabama', 3, 2), 2: ('Alaska', 4, 4), 3: ('Arizona', 4, 3), 4: ('Arkansas', 3, 2),
    5: ('California', 4, 4), 6: ('Colorado', 4, 3), 7: ('Connecticut', 1, 1),
    8: ('Delaware', 3, 1), 9: ('District of Columbia', 3, 1), 10: ('Florida', 3, 1),
    11: ('Georgia', 3, 1), 12: ('Hawaii', 4, 4), 13: ('Idaho', 4, 3), 14: ('Illinois', 2, 2),
    15: ('Indiana', 2, 1), 16: ('Iowa', 2, 2), 17: ('Kansas', 2, 2), 18: ('Kentucky', 3, 1),
    19: ('Louisiana', 3, 2), 20: ('Maine', 1, 1), 21: ('Maryland', 3, 1), 
    22: ('Massachusetts', 1, 1), 23: ('Michigan', 2, 1), 24: ('Minnesota', 2, 2),
    25: ('Mississippi', 3, 2), 26: ('Missouri', 2, 2), 27: ('Montana', 4, 3),
    28: ('Nebraska', 2, 1), 29: ('Nevada', 4, 3), 30: ('New Hampshire', 1, 1),
    31: ('New Jersey', 1, 1), 32: ('New Mexico', 4, 3), 33: ('New York', 1, 1),
    34: ('North Carolina', 3, 1), 35: ('North Dakota', 2, 2), 36: ('Ohio', 2, 1),
    37: ('Oklahoma', 3, 2), 38: ('Oregon', 4, 4), 39: ('Pennsylvania', 1, 1),
    40: ('Rhode Island', 1, 1), 41: ('South Carolina', 3, 1), 42: ('South Dakota', 2, 2),
    43: ('Tennessee', 3, 2), 44: ('Texas', 3, 2), 45: ('Utah', 4, 3), 
    46: ('Vermont', 1, 1), 47: ('Virginia', 3, 1), 48: ('Washington', 4, 4),
    49: ('West Virginia', 3, 1), 50: ('Wisconsin', 2, 2), 51: ('Wyoming', 4, 3)
}

combined_data_2point5[['state', 'region', 'tz']] = combined_data_2point5['Q60'].map(Q60_mapping).apply(pd.Series)

# Assign time of day based on 'tz' and 'StartHour'
def assign_time_of_day(row):
    if row['tz'] == 1:
        if 4 <= row['StartHour'] <= 9:
            return 'morning'
        elif 10 <= row['StartHour'] <= 12:
            return 'lunch'
        elif 13 <= row['StartHour'] <= 16:
            return 'afternoon'
        elif 17 <= row['StartHour'] <= 19:
            return 'earlyevening'
        elif 20 <= row['StartHour'] <= 23:
            return 'lateevening'
        elif 0 <= row['StartHour'] <= 3 or row['StartHour'] == 24:
            return 'night'
    elif row['tz'] == 2:
        if 5 <= row['StartHour'] <= 10:
            return 'morning'
        elif 11 <= row['StartHour'] <= 13:
            return 'lunch'
        elif 14 <= row['StartHour'] <= 17:
            return 'afternoon'
        elif 18 <= row['StartHour'] <= 20:
            return 'earlyevening'
        elif 21 <= row['StartHour'] <= 24 or row['StartHour'] == 0:
            return 'lateevening'
        elif 1 <= row['StartHour'] <= 4:
            return 'night'
    elif row['tz'] == 3:
        if 6 <= row['StartHour'] <= 11:
            return 'morning'
        elif 12 <= row['StartHour'] <= 14:
            return 'lunch'
        elif 15 <= row['StartHour'] <= 18:
            return 'afternoon'
        elif 19 <= row['StartHour'] <= 21:
            return 'earlyevening'
        elif 22 <= row['StartHour'] <= 1:
            return 'lateevening'
        elif 2 <= row['StartHour'] <= 5:
            return 'night'
    elif row['tz'] == 4:
        if 7 <= row['StartHour'] <= 12:
            return 'morning'
        elif 13 <= row['StartHour'] <= 15:
            return 'lunch'
        elif 16 <= row['StartHour'] <= 19:
            return 'afternoon'
        elif 20 <= row['StartHour'] <= 22:
            return 'earlyevening'
        elif 23 <= row['StartHour'] <= 2:
            return 'lateevening'
        elif 3 <= row['StartHour'] <= 6:
            return 'night'

combined_data_2point5['timeofday'] = combined_data_2point5.apply(assign_time_of_day, axis=1)


In [8]:
# Initialize time-of-day columns
combined_data_2point5['morning'] = 0
combined_data_2point5['lunch'] = 0
combined_data_2point5['afternoon'] = 0
combined_data_2point5['earlyevening'] = 0
combined_data_2point5['lateevening'] = 0
combined_data_2point5['night'] = 0

# Assign time-of-day based on `tz` and `StartHour`
def assign_time_blocks(row):
    if row['tz'] == 2:
        if 5 <= row['StartHour'] <= 10:
            row['morning'] = 1
        elif 11 <= row['StartHour'] <= 13:
            row['lunch'] = 1
        elif 14 <= row['StartHour'] <= 17:
            row['afternoon'] = 1
        elif 18 <= row['StartHour'] <= 20:
            row['earlyevening'] = 1
        elif 21 <= row['StartHour'] <= 24:
            row['lateevening'] = 1
        elif 1 <= row['StartHour'] <= 4:
            row['night'] = 1
    elif row['tz'] == 3:
        if 6 <= row['StartHour'] <= 11:
            row['morning'] = 1
        elif 12 <= row['StartHour'] <= 14:
            row['lunch'] = 1
        elif 15 <= row['StartHour'] <= 18:
            row['afternoon'] = 1
        elif 19 <= row['StartHour'] <= 21:
            row['earlyevening'] = 1
        elif row['StartHour'] in [22, 23, 24, 1]:
            row['lateevening'] = 1
        elif 2 <= row['StartHour'] <= 5:
            row['night'] = 1
    elif row['tz'] == 4:
        if 7 <= row['StartHour'] <= 12:
            row['morning'] = 1
        elif 13 <= row['StartHour'] <= 15:
            row['lunch'] = 1
        elif 16 <= row['StartHour'] <= 19:
            row['afternoon'] = 1
        elif 20 <= row['StartHour'] <= 22:
            row['earlyevening'] = 1
        elif row['StartHour'] in [21, 22, 23, 24, 1, 2]:
            row['lateevening'] = 1
        elif 3 <= row['StartHour'] <= 6:
            row['night'] = 1
    return row

combined_data_2point5 = combined_data_2point5.apply(assign_time_blocks, axis=1)


In [9]:
issue_to_month = {
    1: 5, 2: 6, 3: 7, 4: 8, 5: 9, 6: 10, 7: 11, 8: 12, 9: 1,
    10: 2, 11: 3, 12: 4, 13: 5, 14: 6, 15: 7, 16: 8, 17: 9, 18: 10,
    19: 11, 20: 12, 21: 1, 22: 2, 23: 3, 24: 4, 25: 5, 26: 6, 27: 7,
    28: 8, 29: 9, 30: 10, 31: 11, 32: 12, 33: 1
}
combined_data_2point5['month'] = combined_data_2point5['Issue'].map(issue_to_month)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\1128438708.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['month'] = combined_data_2point5['Issue'].map(issue_to_month)


In [10]:
combined_data_2point5['age'] = combined_data_2point5['Q52'].replace({1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 6})

In [11]:
combined_data_2point5['edu'] = pd.cut(
    combined_data_2point5['Q61'], 
    bins=[-float('inf'), 2, 4, float('inf')],
    labels=[1, 2, 3]
).astype(float)

In [12]:
combined_data_2point5['gender'] = combined_data_2point5['Q51']

In [13]:
combined_data_2point5['race'] = combined_data_2point5['Q64'].apply(lambda x: 1 if x == 1 else (2 if x == 2 else (3 if x == 3 else 4)))

In [14]:
combined_data_2point5['inc'] = pd.cut(
    combined_data_2point5['Q62'],
    bins=[-float('inf'), 2, 5, float('inf')],
    labels=[1, 2, 3]
).astype(float)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\1680322895.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['inc'] = pd.cut(


In [15]:
combined_data_2point5['lowinc'] = (combined_data_2point5['inc'] == 1).astype(int)
combined_data_2point5['medinc'] = (combined_data_2point5['inc'] == 2).astype(int)
combined_data_2point5['hiinc'] = (combined_data_2point5['inc'] == 3).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\2418350560.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['lowinc'] = (combined_data_2point5['inc'] == 1).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\2418350560.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['medinc'] = (combined_data_2point5['inc'] == 2).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\2418350560.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually

In [16]:
combined_data_2point5['age18_24'] = (combined_data_2point5['age'] == 1).astype(int)
combined_data_2point5['age25_34'] = (combined_data_2point5['age'] == 2).astype(int)
combined_data_2point5['age35_44'] = (combined_data_2point5['age'] == 3).astype(int)
combined_data_2point5['age45_54'] = (combined_data_2point5['age'] == 4).astype(int)
combined_data_2point5['age55_64'] = (combined_data_2point5['age'] == 5).astype(int)
combined_data_2point5['age65p'] = (combined_data_2point5['age'] == 6).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\910422865.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['age18_24'] = (combined_data_2point5['age'] == 1).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\910422865.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['age25_34'] = (combined_data_2point5['age'] == 2).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\910422865.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

In [17]:
for value, name in [
    (1, 'inc19'), (2, 'inc20'), (3, 'inc40'), (4, 'inc60'), (5, 'inc80'), 
    (6, 'inc100'), (7, 'inc120'), (8, 'inc140'), (9, 'inc160')
]:
    combined_data_2point5[name] = (combined_data_2point5['Q62'] == value).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3951717409.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5[name] = (combined_data_2point5['Q62'] == value).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3951717409.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5[name] = (combined_data_2point5['Q62'] == value).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3951717409.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually

In [18]:
combined_data_2point5['hs'] = (combined_data_2point5['edu'] == 1).astype(int)
combined_data_2point5['somecoll'] = (combined_data_2point5['edu'] == 2).astype(int)
combined_data_2point5['college'] = (combined_data_2point5['edu'] == 3).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\2753457134.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['hs'] = (combined_data_2point5['edu'] == 1).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\2753457134.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['somecoll'] = (combined_data_2point5['edu'] == 2).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\2753457134.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually t

In [19]:
combined_data_2point5['female'] = (combined_data_2point5['Q51'] == 2).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3051381937.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['female'] = (combined_data_2point5['Q51'] == 2).astype(int)


In [20]:
combined_data_2point5['white'] = (combined_data_2point5['race'] == 1).astype(int)
combined_data_2point5['black'] = (combined_data_2point5['race'] == 2).astype(int)
combined_data_2point5['otherrace'] = ((combined_data_2point5['race'] == 3) | (combined_data_2point5['race'] == 4)).astype(int)
combined_data_2point5['hispanic'] = (combined_data_2point5['Q63'] > 1).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\1029867607.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['white'] = (combined_data_2point5['race'] == 1).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\1029867607.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['black'] = (combined_data_2point5['race'] == 2).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\1029867607.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually

In [21]:
combined_data_2point5['northeast'] = (combined_data_2point5['region'] == 1).astype(int)
combined_data_2point5['midwest'] = (combined_data_2point5['region'] == 2).astype(int)
combined_data_2point5['south'] = (combined_data_2point5['region'] == 3).astype(int)
combined_data_2point5['west'] = (combined_data_2point5['region'] == 4).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3280711513.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['northeast'] = (combined_data_2point5['region'] == 1).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3280711513.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['midwest'] = (combined_data_2point5['region'] == 2).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3280711513.py:3: PerformanceWarning: DataFrame is highly fragmented.  This 

In [22]:
# Adjust weight and height
combined_data_2point5['weightlb'] = combined_data_2point5['Q56'].clip(lower=90, upper=490).fillna(172)
combined_data_2point5['height'] = combined_data_2point5['Q57'] + 48

# Calculate BMI
combined_data_2point5['bmi'] = (combined_data_2point5['weightlb'] * 703) / (combined_data_2point5['height'] ** 2)
combined_data_2point5['bmi'] = combined_data_2point5['bmi'].clip(lower=15, upper=40).fillna(27)

# BMI categories
combined_data_2point5['underweight'] = (combined_data_2point5['bmi'] < 18.5).astype(int)
combined_data_2point5['overweight'] = ((combined_data_2point5['bmi'] >= 25) & (combined_data_2point5['bmi'] < 30)).astype(int)
combined_data_2point5['obese'] = (combined_data_2point5['bmi'] >= 30).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3464316014.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['weightlb'] = combined_data_2point5['Q56'].clip(lower=90, upper=490).fillna(172)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3464316014.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['height'] = combined_data_2point5['Q57'] + 48
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3464316014.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is 

In [23]:
combined_data_2point5['primaryshop'] = (combined_data_2point5['Q48'] == 1).astype(int)
combined_data_2point5['farmwork'] = (combined_data_2point5['Q49'] == 1).astype(int)
combined_data_2point5['kids'] = (combined_data_2point5['Q55'] == 1).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\1561155740.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['primaryshop'] = (combined_data_2point5['Q48'] == 1).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\1561155740.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['farmwork'] = (combined_data_2point5['Q49'] == 1).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\1561155740.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is 

In [24]:
combined_data_2point5['hsize'] = combined_data_2point5['Q54']

for value in range(1, 6):
    combined_data_2point5[f'hsize{value}'] = (combined_data_2point5['hsize'] == value).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\623455558.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['hsize'] = combined_data_2point5['Q54']
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\623455558.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5[f'hsize{value}'] = (combined_data_2point5['hsize'] == value).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\623455558.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resul

In [25]:
combined_data_2point5['single'] = (combined_data_2point5['Q53'] == 1).astype(int)
combined_data_2point5['married'] = (combined_data_2point5['Q53'] == 2).astype(int)
combined_data_2point5['omarry'] = (combined_data_2point5['Q53'] > 2).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\357369533.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['single'] = (combined_data_2point5['Q53'] == 1).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\357369533.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['married'] = (combined_data_2point5['Q53'] == 2).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\357369533.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually t

In [26]:
combined_data_2point5['foodstamp'] = (combined_data_2point5['Q59'] == 1).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\272065322.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['foodstamp'] = (combined_data_2point5['Q59'] == 1).astype(int)


In [27]:
combined_data_2point5['conservative'] = combined_data_2point5['Q65'].apply(lambda x: x if x < 6 else 3)
combined_data_2point5['dkideology'] = (combined_data_2point5['Q65'] == 6).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3984763732.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['conservative'] = combined_data_2point5['Q65'].apply(lambda x: x if x < 6 else 3)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3984763732.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['dkideology'] = (combined_data_2point5['Q65'] == 6).astype(int)


In [28]:
combined_data_2point5['democrat'] = (combined_data_2point5['Q66'] == 1).astype(int)
combined_data_2point5['repub'] = (combined_data_2point5['Q66'] == 2).astype(int)
combined_data_2point5['tea'] = (combined_data_2point5['Q66'] == 3).astype(int)
combined_data_2point5['indep'] = (combined_data_2point5['Q66'] == 4).astype(int)
combined_data_2point5['oparty'] = (combined_data_2point5['Q66'] == 5).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\2413255877.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['democrat'] = (combined_data_2point5['Q66'] == 1).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\2413255877.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['repub'] = (combined_data_2point5['Q66'] == 2).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\2413255877.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

In [29]:
combined_data_2point5['foodpois'] = (combined_data_2point5['Q14'] == 1).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3371667187.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5['foodpois'] = (combined_data_2point5['Q14'] == 1).astype(int)


In [30]:
day_map = {
    'Monday': 'dow1', 'Tuesday': 'dow2', 'Wednesday': 'dow3',
    'Thursday': 'dow4', 'Friday': 'dow5', 'Saturday': 'dow6', 'Sunday': 'dow7'
}
for day, column in day_map.items():
    combined_data_2point5[column] = (combined_data_2point5['DayofWeek'] == day).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\1056184398.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5[column] = (combined_data_2point5['DayofWeek'] == day).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\1056184398.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5[column] = (combined_data_2point5['DayofWeek'] == day).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\1056184398.py:6: PerformanceWarning: DataFrame is highly fragmented.  Thi

In [31]:
for i in range(1, 5):
    combined_data_2point5[f'long{i}'] = (combined_data_2point5['length2'] == i).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3145537243.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5[f'long{i}'] = (combined_data_2point5['length2'] == i).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3145537243.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5[f'long{i}'] = (combined_data_2point5['length2'] == i).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3145537243.py:2: PerformanceWarning: DataFrame is highly fragmented.  Thi

In [32]:
for i in range(1, 13):
    combined_data_2point5[f'm{i}'] = (combined_data_2point5['month'] == i).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3620911845.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5[f'm{i}'] = (combined_data_2point5['month'] == i).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3620911845.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5[f'm{i}'] = (combined_data_2point5['month'] == i).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\3620911845.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usual

In [33]:
for i in range(1, 5):
    combined_data_2point5[f'region{i}'] = (combined_data_2point5['region'] == i).astype(int)

C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\1531473743.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5[f'region{i}'] = (combined_data_2point5['region'] == i).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\1531473743.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_data_2point5[f'region{i}'] = (combined_data_2point5['region'] == i).astype(int)
C:\Users\schmiess\AppData\Local\Temp\ipykernel_10352\1531473743.py:2: PerformanceWarning: DataFrame is highly fragmented.  T

In [34]:
labels = [
    'madcow', 'bse', 'antibiotic', 'hormone', 'ecoli', 'salmonella', 
    'gmo', 'pinkslime', 'lftgb', 'gestcrate', 'faw', 'cages', 'clone', 
    'greengas', 'swineflu', 'bircombined_data_2point5lu'
]

# Apply labels to Q12 variables
Q12_labels = {f'Q12_{i+1}': label for i, label in enumerate(labels)}
combined_data_2point5 = combined_data_2point5.rename(columns=Q12_labels)

# Apply labels to Q13 variables
Q13_labels = {f'Q13_{i+1}': label for i, label in enumerate(labels)}
combined_data_2point5 = combined_data_2point5.rename(columns=Q13_labels)

In [35]:
Q10_labels = {
    'Q10_1': 'finding_affordable_foods',
    'Q10_2': 'avoiding_pesticides_hormones_antibiotics',
    'Q10_3': 'finding_Quick_alternatives',
    'Q10_4': 'finding_foods_for_children',
    'Q10_5': 'losing_weight',
    'Q10_6': 'finding_time_to_cook',
    'Q10_7': 'avoiding_certain_nutrients'
}

combined_data_2point5 = combined_data_2point5.rename(columns=Q10_labels)

In [36]:
Q9_labels = {
    'Q9_1': 'plan_buy_more_beef',
    'Q9_2': 'plan_buy_more_chicken',
    'Q9_3': 'plan_buy_more_pork',
    'Q9_4': 'plan_eat_out_more',
    'Q9_5': 'expect_higher_beef_price',
    'Q9_6': 'expect_higher_pork_price',
    'Q9_7': 'expect_higher_chicken_price'
}

combined_data_2point5 = combined_data_2point5.rename(columns=Q9_labels)

In [37]:
# List of renamed columns
renamed_columns = list(Q9_labels.values())

# % Disagree Flags
for col in renamed_columns:
    flag_name = col.replace("plan_", "disagree_plan_").replace("expect_", "disagree_expect_")
    combined_data_2point5[f'{flag_name}'] = combined_data_2point5[col].apply(lambda x: 1 if x in [1, 2] else 0)

# % Agree Flags
for col in renamed_columns:
    flag_name = col.replace("plan_", "agree_plan_").replace("expect_", "agree_expect_")
    combined_data_2point5[f'{flag_name}'] = combined_data_2point5[col].apply(lambda x: 1 if x in [4, 5] else 0)

In [38]:
Q9da_labels = {
    'Q9da_1': 'disagree_plan_buy_more_beef',
    'Q9da_2': 'disagree_plan_buy_more_chicken',
    'Q9da_3': 'disagree_plan_buy_more_pork',
    'Q9da_4': 'disagree_plan_eat_out_more',
    'Q9da_5': 'disagree_expect_higher_beef_price',
    'Q9da_6': 'disagree_expect_higher_pork_price',
    'Q9da_7': 'disagree_expect_higher_chicken_price'
}

Q9a_labels = {
    'Q9a_1': 'agree_plan_buy_more_beef',
    'Q9a_2': 'agree_plan_buy_more_chicken',
    'Q9a_3': 'agree_plan_buy_more_pork',
    'Q9a_4': 'agree_plan_eat_out_more',
    'Q9a_5': 'agree_expect_higher_beef_price',
    'Q9a_6': 'agree_expect_higher_pork_price',
    'Q9a_7': 'agree_expect_higher_chicken_price'
}

combined_data_2point5 = combined_data_2point5.rename(columns={**Q9da_labels, **Q9a_labels})

In [39]:
# For Q4
lo_values = {1: 0, 2: 20, 3: 40, 4: 60, 5: 80, 6: 100, 7: 120, 8: 140, 9: 160}
hhi_values = {1: 20, 2: 39, 3: 59, 4: 79, 5: 99, 6: 119, 7: 139, 8: 159, 9: None}
combined_data_2point5['lo'] = combined_data_2point5['Q4'].map(lo_values)
combined_data_2point5['hhi'] = combined_data_2point5['Q4'].map(hhi_values)

# For Q5
lo1_values = lo_values  # Same mapping as lo
hi1_values = hhi_values  # Same mapping as hhi
combined_data_2point5['lo1'] = combined_data_2point5['Q5'].map(lo1_values)
combined_data_2point5['hi1'] = combined_data_2point5['Q5'].map(hi1_values)

# For Q7
lo2_values = {1: None, 2: -7.5, 3: -2.5, 4: 2.5, 5: 7.5}
hi2_values = {1: -7.5, 2: -2.5, 3: 2.5, 4: 7.5, 5: None}
combined_data_2point5['lo2'] = combined_data_2point5['Q7'].map(lo2_values)
combined_data_2point5['hi2'] = combined_data_2point5['Q7'].map(hi2_values)

# For Q8
lo3_values = lo2_values  # Same mapping as lo2
hi3_values = hi2_values  # Same mapping as hi2
combined_data_2point5['lo3'] = combined_data_2point5['Q8'].map(lo3_values)
combined_data_2point5['hi3'] = combined_data_2point5['Q8'].map(hi3_values)

In [40]:
low_flags = {
    'Q2_1_Group': 'natlo', 'Q2_2_Group': 'tastelo', 'Q2_3_Group': 'pricelo', 
    'Q2_4_Group': 'safelo', 'Q2_5_Group': 'convenlo', 'Q2_6_Group': 'nutlo',
    'Q2_7_Group': 'novello', 'Q2_8_Group': 'originlo', 'Q2_9_Group': 'fairlo',
    'Q2_10_Group': 'appearlo', 'Q2_11_Group': 'envlo', 'Q2_12_Group': 'awlo'
}

high_flags = {
    'Q2_1_Group': 'nathi', 'Q2_2_Group': 'tastehi', 'Q2_3_Group': 'pricehi', 
    'Q2_4_Group': 'safehi', 'Q2_5_Group': 'convenhi', 'Q2_6_Group': 'nuthi',
    'Q2_7_Group': 'novelhi', 'Q2_8_Group': 'originhi', 'Q2_9_Group': 'fairhi',
    'Q2_10_Group': 'appearhi', 'Q2_11_Group': 'envhi', 'Q2_12_Group': 'awhi'
}

# Low Flags
for Q, flag in low_flags.items():
    combined_data_2point5[flag] = (combined_data_2point5[Q] == 0).astype(int)

# High Flags
for Q, flag in high_flags.items():
    combined_data_2point5[flag] = (combined_data_2point5[Q] == 1).astype(int)

In [41]:
combined_data_2point5['nat'] = combined_data_2point5['natlo'] - combined_data_2point5['nathi']
combined_data_2point5['taste'] = combined_data_2point5['tastelo'] - combined_data_2point5['tastehi']
combined_data_2point5['price1'] = combined_data_2point5['pricelo'] - combined_data_2point5['pricehi']
combined_data_2point5['safe'] = combined_data_2point5['safelo'] - combined_data_2point5['safehi']
combined_data_2point5['conven'] = combined_data_2point5['convenlo'] - combined_data_2point5['convenhi']
combined_data_2point5['nut'] = combined_data_2point5['nutlo'] - combined_data_2point5['nuthi']
combined_data_2point5['novel'] = combined_data_2point5['novello'] - combined_data_2point5['novelhi']
combined_data_2point5['origin'] = combined_data_2point5['originlo'] - combined_data_2point5['originhi']
combined_data_2point5['fair'] = combined_data_2point5['fairlo'] - combined_data_2point5['fairhi']
combined_data_2point5['appear'] = combined_data_2point5['appearlo'] - combined_data_2point5['appearhi']
combined_data_2point5['env'] = combined_data_2point5['envlo'] - combined_data_2point5['envhi']
combined_data_2point5['aw'] = combined_data_2point5['awlo'] - combined_data_2point5['awhi']

In [42]:
combined_data_2point5['illness'] = (combined_data_2point5['Q14'] == 1).astype(int)
combined_data_2point5['vegetarian'] = (combined_data_2point5['Q47'] == 1).astype(int)

In [43]:
combined_data_2point5['totlo'] = combined_data_2point5['lo'] + combined_data_2point5['lo1']
combined_data_2point5['tothi'] = combined_data_2point5['hhi'] + combined_data_2point5['hi1']

In [44]:
for i in range(2, 34):
    combined_data_2point5[f'i{i}'] = (combined_data_2point5['Issue'] == i).astype(int)

In [45]:
# Generate list of Q16 to Q42 variable names
q_vars = [f'Q{i}_1' for i in range(16, 43)]

# Combine the list of variables to keep
keep_vars = [
    # Demographic and socioeconomic variables
    'age', 'edu', 'gender', 'race', 'hispanic', 'lowinc', 'medinc', 'hiinc',
    'hsize', 'kids', 'region', 'northeast', 'midwest', 'south', 'west',

    # Ideological and political variables
    'conservative', 'dkideology', 'democrat', 'repub', 'tea', 'indep', 'oparty',

    # Behavioral and lifestyle variables
    'vegetarian', 'illness', 'primaryshop', 'farmwork',

    # Choice and preference variables
    'agree_plan_buy_more_beef', 'disagree_plan_buy_more_beef',
    # (Include other `% Agree` and `% Disagree` variables here)
    'nat', 'taste', 'price1', 'safe', 'conven', 'nut', 'novel', 'origin',
    'fair', 'appear', 'env', 'aw',

    # Time-based and grouping variables
    'month', 'DayofWeek', 'Issue',

    # Calculated metrics
    'bmi', 'underweight', 'overweight', 'obese',

    # ResponseID
    'ResponseID'
] + q_vars  # Add Q16 to Q42 variables

# Subset the dataframe
cleaned_df = combined_data_2point5[keep_vars]

In [46]:
cleaned_df.to_csv("../data/cleaned_df.csv", index=False)

## Choice and price

In [12]:
# Load cleaned data and subset to feasible amount of obs
cleaned_df = pd.read_csv("../data/cleaned_df.csv")
cleaned_df = cleaned_df[cleaned_df['Issue'] <= 3]
pricedat2 = pd.read_excel("../data/pricedat2.xlsx")

In [13]:
# Repeat each row 81 times
cleaned_df_long = cleaned_df.loc[cleaned_df.index.repeat(81)].reset_index(drop=True)

In [14]:
# Create the 'count' column
cleaned_df_long['count'] = np.tile(np.arange(1, 82), len(cleaned_df_long) // 81)

In [15]:
# Create the 'Qnum' column
Qnum_pattern = np.tile(np.repeat(np.arange(1, 10), 9), len(cleaned_df_long) // (9 * 9))
cleaned_df_long['Qnum'] = Qnum_pattern[:len(cleaned_df_long)]

In [16]:
# Create the 'option' column
option_pattern = np.tile(np.arange(1, 10), len(cleaned_df_long) // 9)
cleaned_df_long['option'] = option_pattern[:len(cleaned_df_long)]

In [17]:
# Create the 'series' column 
cleaned_df_long['series'] = np.nan  # Initialize with NaN
cleaned_df_long.loc[cleaned_df_long['Q16_1'].notna(), 'series'] = cleaned_df_long['count']
cleaned_df_long.loc[cleaned_df_long['Q25_1'].notna(), 'series'] = cleaned_df_long['count'] + 81
cleaned_df_long.loc[cleaned_df_long['Q34_1'].notna(), 'series'] = cleaned_df_long['count'] + 162

In [18]:
# Alter the 'Qnum' column 
cleaned_df_long.loc[cleaned_df_long['Q16_1'].notna(), 'Qnum'] = cleaned_df_long['Qnum']
cleaned_df_long.loc[cleaned_df_long['Q25_1'].notna(), 'Qnum'] = cleaned_df_long['Qnum'] + 9
cleaned_df_long.loc[cleaned_df_long['Q34_1'].notna(), 'Qnum'] = cleaned_df_long['Qnum'] + 18

In [19]:
# Merge to get 'price'
cleaned_df_long = cleaned_df_long.merge(pricedat2[['series', 'Price']], on='series', how='left')
cleaned_df_long = cleaned_df_long.rename(columns={'Price': 'price'})

In [20]:
# Create dummies for ASCs
columns = ['burger', 'steak', 'chop', 'ham', 'breast', 'wing', 'bean', 'pasta', 'none']
for i, col in enumerate(columns):
    cleaned_df_long[col] = (cleaned_df_long['option'] == (i + 1)).astype(int)

In [21]:
# Create choice variable
# Initialize the 'choice' column with 0
cleaned_df_long['choice'] = 0

# Define the Question Groups and their Qnum ranges
Question_Groups = [
    (range(16, 25), range(1, 10)),    # Q16_1-Q24_1 corresponds to Qnum 1-9
    (range(25, 34), range(10, 19)),  # Q25_1-Q33_1 corresponds to Qnum 10-18
    (range(34, 43), range(19, 28))   # Q34_1-Q42_1 corresponds to Qnum 19-27
]

# Loop through each Question Group
for Question_range, Qnum_range in Question_Groups:
    for Q_col, Qnum in zip(Question_range, Qnum_range):
        col_name = f"Q{Q_col}_1"  # Generate column names dynamically (e.g., Q16_1, Q25_1)
        # Update 'choice' based on the conditions
        cleaned_df_long.loc[
            (cleaned_df_long['Qnum'] == Qnum) &  # Match Qnum
            cleaned_df_long[col_name].notna() &  # Ensure the Question column is not missing
            (cleaned_df_long[col_name] == cleaned_df_long['option']),  # Match option
            'choice'
        ] = 1

In [22]:
# Generate list of Q16 to Q42 variable names
q_vars = [f'Q{i}_1' for i in range(16, 43)]
cleaned_df_long = cleaned_df_long.drop(columns=q_vars)

## Analysis

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Multinomial Logit